# <font color='#DE3163'>U</font><font color='#6495ED'>n</font><font color='#FF7F50'>i</font><font color='#CCCCFF'>q</font><font color='#FFC300'>u</font><font color='#40E0D0'>e</font> > Random</font>
**Generating a unique networking schedule using object oriented programming in Python**  
Anne Bode

## Import Libraries

In [2]:
import random
import copy
import pandas as pd

## Create Objects: Student & Lunch Group

In [200]:
# create new object type: student
class student:
    
    '''
    New object type, "student"
    Takes email and name as inputs
    Keeps track of which other students the student has met
    Meets other students, and forgets them when necessary (when student is removed from networking group)
    '''
    
    def __init__(self, email, name):
        self.email = email
        self.name = name
        self.students_met = [] # list of student objects
    
    def __str__(self):
        return self.email, self.name
    
    def get_students_met(self):
        return self.students_met
        
    def meet_someone(self, other_student):
        if other_student != self:
            self.students_met.append(other_student)
            other_student.students_met.append(self)
    
    def forget_someone(self, other_student):
        self.students_met.remove(other_student) # remove other student from students_met (first occurrence)
        other_student.students_met.remove(self) # remove self from other student's students_met (first occurrence)

In [201]:
# create new object type: lunch group
class lunch_group:
    
    '''
    New object type, "lunch_group"
    No inputs needed to create new instance
    Tracks who is a member of the group, and who the members of the group have met
    Returns number of members
    Updates values as necessary when new student is added to group or student is removed
    '''  
        
    def __init__(self):
        self.student_list = []
        self.students_met = []
    
    def get_student_list(self):
        return self.student_list
    
    def get_student_list_print(self):
        return [s.email for s in self.student_list], [s.name for s in self.student_list]
    
    def get_students_met(self):
        return self.students_met
    
    def get_students_met_print(self):
        return [s.email for s in self.students_met], [s.name for s in self.students_met]
    
    def get_number_members(self):
        return len(self.student_list)
    
    # create list of all students met across all students in the lunch group
    # if members have been removed or added, we must update for the group
    def update_students_met(self):
        self.students_met = []
        for s in self.student_list:
            for t in s.students_met:
                if t not in self.students_met:
                    self.students_met.append(t)

    # add student to the group, all students in the group will now meet eachother
    def add_student(self, new_student):
        if new_student not in self.student_list:
            self.student_list.append(new_student)
            for s in self.student_list:
                new_student.meet_someone(s)
            self.update_students_met()
    
    # remove student from group, students will now "forget" this student
    # if student has previously met removed student, only this one meeting will be "forgotten"
    def remove_student(self, removed_student):
        self.student_list.remove(removed_student)
        for s in self.student_list:
            s.forget_someone(removed_student) # removes first occurrence from both students' students_met list


## Weekly Plan Functions

In [202]:
def calc_groups(n, group_size):
    
    '''
    calculate number of groups, given number of participants and desired group size
    inputs: n = number of participants | group_size = desired group size
    output: number of groups, remainder of n % group size, and code to say whether there should be smaller/larger groups
    '''
    
    num_groups = int(n/group_size)
    remainder = n % group_size
    group_code = 'normal'
    
    # if our remainder is > 50% of the desired group size, we will have one extra, smaller group
    if remainder > group_size/2:
        num_groups +=1
        group_code = 'extra_group'
    
    # if our remainder is < 50% of the desired group size but > 0, our last n=remainder groups will have one extra member
    elif remainder > 0:
        group_code = 'bigger_groups'
        
    return num_groups, remainder, group_code

In [203]:
def random_schedule(students, group_size, num_weeks):
    
    '''
    Generates a weekly schedule of group meetings, just relying on randomness to create groupings
    Takes in list of students, group size, and number of weeks of the networking program
    Returns a weekly plan, with appropriate sized groups meeting for the specified number of weeks
    '''
    
    # initiate variables
    weekly_plan = {}
    weeks = ['week'+ str(i) for i in range(1,num_weeks+1)]
    num_participants = len(students)
    
    # calculate how many groups we should create, depending on number of participants and desired group size
    num_groups, remainder, group_code = calc_groups(num_participants, group_size)
    
    for w in weeks:
        weekly_plan[w] = []
        students_unassigned = students[:] # create copy of students to keep track of who has been assigned a group
        
        for i in range(1, num_groups+1):
            # initiate new instance of lunch_group
            new_group = lunch_group()
            
            # determine size of group, using results from above
            if group_code == 'extra_group' and i == num_groups:
                size = remainder
            elif group_code == 'bigger_groups' and i > (num_groups - remainder):
                size = group_size + 1
            else:
                size = group_size
            
            # while our new group has fewer members than our specified size,
            # randomly add an unassigned student to the group and remove student from unassigned list
            while new_group.get_number_members() < size:
                new_student = random.choice(students_unassigned)
                new_group.add_student(new_student)
                students_unassigned.remove(new_student)
            
            # when specified size has been reached, add new group to this week's entry in our dictionary
            weekly_plan[w].append(new_group)
            
    return weekly_plan

In [204]:
def unique_schedule(students, group_size, num_weeks):
    
    '''
    Generates a weekly schedule of group meetings
    Takes into account who students have already met, to try to create groupings based on uniqueness
    Takes in list of students, group size, and number of weeks of the networking program
    Returns a weekly plan, with appropriate sized groups meeting for the specified number of weeks
    '''

    # initiate variables
    weekly_plan = {}
    weeks = ['week'+ str(i) for i in range(1,num_weeks+1)]
    num_participants = len(students)

    # calculate how many groups we should create, depending on number of participants and desired group size
    num_groups, remainder, group_code = calc_groups(num_participants, group_size)
        
    for w in weeks:
        weekly_plan[w] = []
        students_unassigned = students[:] # create copy of students to keep track of who has been assigned a group
        
        for i in range(1, num_groups+1):
            
            # for each group, create a copy of the currently unassigned students
            # we will use this to keep track of which unassigned students are still eligible to be added to our new group
            group_options = students_unassigned[:]

            # initiate new instance of lunch_group
            new_group = lunch_group()
            
            # create a list of the indices of the other groups we have created thus far
            other_groups = [i for i in range(len(weekly_plan[w]))]
            
            # determine size of group, using results from above
            if group_code == 'extra_group' and i == num_groups:
                size = remainder
            elif group_code == 'bigger_groups' and i > (num_groups - remainder):
                size = group_size + 1
            else:
                size = group_size
            
            # while our new group has fewer members than our specified size, and eligible unassigned students > 0
            while new_group.get_number_members() < size and len(group_options) > 0:
                
                # choose new student at random from eligible list
                # if new student has not met any member of group, add to group and remove from both lists
                # otherwise, remove from just eligible list
                new_student = random.choice(group_options)
                if new_student not in new_group.get_students_met():
                    new_group.add_student(new_student)
                    students_unassigned.remove(new_student)
                    group_options.remove(new_student)
                else:
                    group_options.remove(new_student)
            
            # if we run out of eligible unassigned students and haven't reached our specified group size
            if new_group.get_number_members() < size:
                
                # if we are more than 1 away from our desired size, try to steal students from other groups
                # if we have created other groups already this week, loop through them
                # try to find a member who has not met anyone in our new group, and steal them
                if new_group.get_number_members() < (size - 1):
                    
                    while new_group.get_number_members() < size and len(other_groups) > 0:
                        
                            other_group_index = random.choice(other_groups)
                            other_groups.remove(other_group_index)
                            
                            other_group = weekly_plan[w][other_group_index]
                            other_group_size = other_group.get_number_members()
                            
                            for other_student in other_group.get_student_list():
                                if other_student not in new_group.get_students_met():                                        
                                    new_group.add_student(other_student)
                                    other_group.remove_student(other_student)

                                    # now we have to replace the student we removed from their old group
                                    other_group_options = students_unassigned[:]
                                    while other_group.get_number_members() < other_group_size and len(other_group_options) > 0:
                                        other_new_student = random.choice(other_group_options)
                                        if other_new_student not in other_group.get_students_met():
                                            other_group.add_student(other_new_student)
                                            students_unassigned.remove(other_new_student)
                                            other_group_options.remove(other_new_student)
                                        else:
                                            other_group_options.remove(other_new_student)

                                    # if no more eligible students remain, randomly choose from the unassigned students
                                    # to replace the one we took from this other group
                                    if other_group.get_number_members() < other_group_size:
                                        other_new_student = random.choice(students_unassigned)
                                        other_group.add_student(other_new_student)
                                        students_unassigned.remove(other_new_student)

                                    # move on to the next group (don't want to take more than 1 person from a pre-existing group)
                                    break
                                    
            # if after all this looping we STILL haven't reached our specified size, choose randomly from unassigned students
            if new_group.get_number_members() < size:                
                while new_group.get_number_members() < size:
                    new_student = random.choice(students_unassigned)
                    new_group.add_student(new_student)
                    students_unassigned.remove(new_student)
            
            # when specified size has been reached, add new group to this week's entry in our dictionary
            weekly_plan[w].append(new_group)
            
    return weekly_plan

In [205]:
def weekly_plan_df(students, weekly_plan):
    
    '''
    convert weekly plan into two dataframes for easy to read schedules
    inputs: list of students, weekly plan
    output: (df of names/emails per group per week, df of a student schedule table)
    '''
    
    # initate empty dicts for our dataframes
    dict_df = {}
    dict_student_sched = {}
    
    # generate list of names and emails, pulled from our student list
    names, emails = [s.name for s in students], [s.email for s in students]
    
    # generate keys for our dicts
    weeks = ['week' + str(i) for i in range(1,len(weekly_plan)+1)]
    
    # loop through our weeks, names, and groups to create our two dicts
    for w in weeks:
        dict_df[w]={}
        dict_student_sched[w]={}
        for n in names:
            dict_student_sched[w][n] = 0
        for i in range(len(weekly_plan[w])):
            group = weekly_plan[w][i]
            dict_df[w]['group'+str(i+1)] = group.get_student_list_print()
            for s in group.get_student_list():
                dict_student_sched[w][s.name] = i+1
        
    # convert dicts to dfs
    # add email to our student schedule df
    df = pd.DataFrame.from_dict(dict_df)
    df_student_sched = pd.DataFrame.from_dict(dict_student_sched)
    df_student_sched.insert(loc=0, column='Email', value=emails)
    
    return df, df_student_sched

## Generate Weekly Plans

In [3]:
# load data (hiding data to post on GitHub)
participant_list = pd.read_csv('Participant List.csv')

In [207]:
# initiate shared variables
emails = list(participant_list['Email'])
names = list(participant_list['Name'])
students = [student(e,n) for e,n in zip(emails, names)]

### Random Version

In [208]:
# create deep copy of our student list, so we can mutate new version of the student objects
random_student_list = copy.deepcopy(students)

# generate weekly plan
random_weekly_plan = random_schedule(random_student_list, 4, 6)

### Unique Version

In [209]:
# create deep copy of our student list, so we can mutate new version of the student objects
unique_student_list = copy.deepcopy(students)

# generate weekly plan
unique_weekly_plan = unique_schedule(unique_student_list, 4, 6)

In [210]:
## uncomment to save to csv
#plan, student_schedule = weekly_plan_df(unique_student_list, unique_weekly_plan)
#plan.to_csv('Weekly Plan 2.csv')
#student_schedule.to_csv('Student Schedule 2.csv')
#print('done')

## Check Performance

### Random Version: Average Number of People Met, per Student

In [211]:
num_met = []
for s in random_student_list:
    num_met.append(len(set(s.get_students_met())))
sum(num_met)/len(num_met)

15.317073170731707

### Unique Version: Average Number of People Met, per Student

In [212]:
num_met = []
for s in unique_student_list:
    num_met.append(len(set(s.get_students_met())))
sum(num_met)/len(num_met)

18.48780487804878

## What does this mean?
In this example we have 41 participants and a desired group size of 4. That means each week there are ten groups, and one group has 5 members. Therefore, if a participant were to only meet unique students each week, they would expect to meet 6 * ((1/10) * 4 + (9/10) * 3) = 18.6 total students.

If we just assigned students randomly each week, in this run of the function they would meet 15.3 students, on average. However, with our unique algorithm, the expected number is 18.5, very close to totally unique!